##  Chameleon/FABRIC L3 Stitching

This notebook demonstrates how to stitch Chameleon experiments to FABRIC.

### Set Chameleon Environment

FABRIC should already be configured in your Jupyter environment. If this is your first time using FABRIC, may need to follow the [Configure Environment](./fabric_examples/fablib_api/configure_environment/configure_environment.ipynb) notebook to complete the configuration.

Set the following environment vars to the values found in your custom Chameleon-openrc.sh file. You can obtain your Chameleon-openrc.sh from Chameleon using these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#the-openstack-rc-script).  The expected password is not your regular Chameleon password. Instead, you need to create a Chameleon CLI password by following these [directions](https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#cli-authentication)

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fablib_local"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from chameleon_utils.chameleon_config import *
load_chameleon_rc_environment(chameleon_rc_file=f"{os.environ['HOME']}/work/fablib_local_private_config/Chameleon-openrc.sh")

    
from fablib_custom.fablib_custom import *
from fablib_common_utils.utils import *
from fablib_common_utils.fabric_fabnet_slice import *

from chameleon_utils.chameleon_fabnet_stitching import *
from chameleon_utils.chameleon_create_servers import *
from chameleon_utils.chameleon_leases import *


Pick a name for the Chameleon and FABRIC resources. A common issue you might encounter is using a name that is already used on Chameleon. One suggestion is to embedded the date/time in the name to increase the likelyhood that the name is unique.

In [ ]:
time_stamp = datetime.now(tz=tz.tzutc()).strftime('%Y%m%d%H%M')

stitch_name=f"pruth_stitch_{time_stamp}"
servers_name=f"pruth_servers_{time_stamp}"

print(f'time_stamp: {time_stamp}')

In [ ]:
server_lease = create_chameleon_server_lease(name=servers_name) #, node_type='compute_skylake')
fabric_net_lease = create_chameleon_stitched_network_lease(name=stitch_name)
#lease = create_chameleon_storage_network_lease(name=lease_name)


In [ ]:
#delete_chameleon_lease(lease_name)

In [ ]:
try:
    create_chameleon_fabnetv4_network(name=stitch_name, lease=fabric_net_lease)
except Exception as e:
    print(f"Exception: {e}")
    raise e

## Add Chameleon Nodes to the FABNet Network

In [ ]:
try:
    server_count = 1
    create_chameleon_servers(name=f"{stitch_name}_servers", 
                              count=server_count, 
                              #node_type='compute_skylake',
                              #de_type='compute_cascadelake_r',
                              image_name='CC-CentOS8-stream', #'CC-Ubuntu20.04',
                              key_name='my_chameleon_key',
                              network_name=stitch_name, 
                              lease=server_lease) 
except Exception as e:
    print(f"Exception: {e}")  


In [ ]:
#get fixed ips
fixed_ips={}
for i in range(1,server_count+1):
    server_name=f"{stitch_name}_servers"
    #server_name=f"{stitch_name}_servers-{i}"
    server_id = chi.server.get_server_id(server_name)
    fixed_ip = chi.server.get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    fixed_ips[server_name]=fixed_ip

for server_name,fixed_ip in fixed_ips.items():
    print(f'{server_name}: {fixed_ip}')

## Add Nodes to FABRIC

In [ ]:
slice = create_fabnet_slice(name=f"{stitch_name}_UTAH_servers", node_count=1, sites=['UTAH'])

## Do something with the Nodes

In [ ]:
threads = []
for node in slice.get_nodes():
    threads.append(node.execute_thread(f'ping -c 5  10.129.130.8'))
    
for thread in threads:
    stdout, stderr = thread.result()
    print(stdout)
    print(stderr)

In [ ]:
for node in slice.get_nodes():
    #stdout, stderr = node.execute(f'ping -c 5 10.129.129.1')
    #print(stdout)
    #print(stderr)
    #stdout, stderr = node.execute(f'ping -c 5 10.129.129.250')
    #print(stdout)
    #print(stderr)
    for server_name,fixed_ip in fixed_ips.items():
        print(f'---- {node.get_name()} -> {server_name} {fixed_ip}---')
        stdout, stderr = node.execute(f'ping -c 5 {fixed_ip}')    
        print(stdout)
        print(stderr)

# Clean Up 

In [ ]:
try:

    delete_chameleon_servers(name=f"{stitch_name}_servers") 
except Exception as e:
    print(f"Exception: {e}")  

In [ ]:
try:
    delete_chameleon_fabnet_stitch(name=name)
except Exception as e:
    print(f"Exception: {e}")


In [ ]:
try:
    delete_fabnet_slice(name=name)
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    fablib = fablib_manager()

    #fablib.show_config()
    fablib.delete_all()
except Exception as e:
    print(f"Exception: {e}") 